In [4]:
%matplotlib widget
import numpy as np

from structure import hydrostat_arm as ha
from structure import draw_nodes
from structure import obstacle

import importlib

# Model Assumptions
1. Every cell maintains a constant volume/area. 
2. Cell edges are not springs, but they are dampers.
3. Muscles may pull, but not push.
4. The base of the arm has pinned boundary conditions for the bottom vertices.

The cell dynamics follow the following model.
$$
\ddot{q} = M^{-1} (F_e + F_i + \hat F - B\dot q - \beta (q, \dot q)) \\
\begin{align*}
\text{subject to } &V = V_0 \\
& F_i \ge 0 \\
& \dot q_\text{base} = 0
\end{align*}
$$

Where $q$ is the $x, y$ position of each vertex, $M$ is a diagonal matrix of vertex masses, $F_e$ are external forces acting on vertices (user defined or collisions), $F_i$ are internal forces generated by muscle activation, $\hat F$ are reaction forces generated by the constant volume constraint, $B$ is a diagonal matrix of damping coefficients for the vertices (viscosity), $\beta$ are linear damping forces from edge contraction/extension, and $V$ and $V_0$ are the current and initial cell volumes.

In [ ]:
importlib.reload(ha)
importlib.reload(draw_nodes)

def straight_arm_maker(height):
    vertices = [[0, 0], [1, 0]]
    for idx in range(height):
        vertices.append([0, (idx+1)*1])
        vertices.append([1, (idx+1)*1])
    vertex_indices = np.arange(len(vertices))
    cells = np.array([[i, j, k] for i, j, k in zip(vertex_indices[:-2], vertex_indices[1:-1], vertex_indices[2:])])
    return np.array(vertices), cells

vertices, cells = straight_arm_maker(10)
vertices = np.array([[0,0], [1, 0], [1, 1]])
cells = np.array([[0, 1, 2]])
# dofs = {0: [1, 0], 1: [0, 0], 2:[1, 0]}
# arm = ha.HydrostatArm(vertices, cells, dofs)
arm = ha.HydrostatArm(vertices, cells)

def odor(x, y):
    x = np.array(x)
    y = np.array(y)
    food_loc = np.array([10, 10])
    covar = np.eye(2)*10
    offset = np.array([[x - food_loc[0], y - food_loc[1]]]).T
    product = (offset.swapaxes(-2, -1) @ np.linalg.inv(covar) @ offset).T
    return np.exp(-0.5 * np.squeeze(product))# / ((2*np.pi)**2 * np.linalg.det(covar))**0.5

# arm.odor_func = odor

obstacle_vertices = np.array([
    [2, 7],
    [2, 0],
    [5, 0],
    [5, 7],
    [3.5, 8]
])
obst = obstacle.ConvexObstacle2D(obstacle_vertices)
arm.add_obstacle(obst)

obstacle_vertices = np.array([
    [2, 16],
    [2, 11],
    [3.5, 10],
    [5, 11],
    [5, 16]
])
obst = obstacle.ConvexObstacle2D(obstacle_vertices)
arm.add_obstacle(obst)


drawer = draw_nodes.NodeDrawer(arm)

In [ ]:
print(arm.cell_volume(arm.vertices[arm.cells[-1]]))

print(np.max(arm.errors))
print(np.max(arm.errors[-1]))
print(arm.constraints().shape)
print(arm.jacobian_derivative().shape)


# Explicit Example

The below sections display an example of the hydrostatic arm calculations more procedurally.

In [7]:
import numpy as np
# Constrained Optimization with inequality constraints

simulation_steps = 2000
dt = 0.01
t = np.arange(0, (simulation_steps+1) * dt, dt)

dim = 2
q0 = np.array([0, 0, 1, 0, 0, 1, 1, 1])
num_nodes = len(q0)//dim
dq0 = np.zeros(num_nodes * dim)
M = np.diag(np.ones(num_nodes * dim))
W = np.linalg.inv(M)
B = np.diag(np.ones(num_nodes * dim))
Q = np.zeros((simulation_steps, num_nodes*dim))
Q[:len(Q)//2,4] = np.ones((simulation_steps//2))/2
Q[len(Q)//2:,4] = -np.ones((simulation_steps//2))/2
F = np.zeros((simulation_steps, 5))
ks = 100
kd = 10

def cell_volume(q):
    return 0.5 * ((q[2] - q[0]) * (q[5] - q[1]) - (q[3] - q[1]) * (q[4] - q[0]))

wall = 1.5
def C(q):
    """Returns an mx1 numpy array of constraints. Each constraint is set equal
    to zero."""
    if q[6] >= wall:
        return np.array([
        cell_volume(q[0:6]) - cell_volume(q0[0:6]),
        q[0],
        q[1],
        cell_volume(q[2:8]) - cell_volume(q0[2:8]),
        q[6] - wall
        ])
    return np.array([
        cell_volume(q[0:6]) - cell_volume(q0[0:6]),
        q[0],
        q[1],
        cell_volume(q[2:8]) - cell_volume(q0[2:8]),
    ])

def jacobian(q):
    """Returns an m x n jacobian where m is the number of constraints and n is
    the length of the state vector. Each element is the partial derivative
    pC_i/pq_j"""
    if q[6] >= wall:
        return np.array([[
            0.5*(q[3] - q[5]),
            0.5*(q[4] - q[2]),
            0.5*(q[5] - q[1]),
            0.5*(q[0] - q[4]),
            0.5*(q[1] - q[3]),
            0.5*(q[2] - q[0]),
            0, 0],

            [1, 0, 0, 0, 0, 0, 0, 0],
            [0, 1, 0, 0, 0, 0, 0, 0],
            
            [0, 0,
            0.5*(q[5] - q[7]),
            0.5*(q[6] - q[4]),
            0.5*(q[7] - q[3]),
            0.5*(q[2] - q[6]),
            0.5*(q[3] - q[5]),
            0.5*(q[4] - q[2])],
            [0, 0, 0, 0, 0, 0, 1, 0]
            ])
    return np.array([[
        0.5*(q[3] - q[5]),
        0.5*(q[4] - q[2]),
        0.5*(q[5] - q[1]),
        0.5*(q[0] - q[4]),
        0.5*(q[1] - q[3]),
        0.5*(q[2] - q[0]),
        0, 0],

        [1, 0, 0, 0, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0, 0, 0],
        
        [0, 0,
        0.5*(q[5] - q[7]),
        0.5*(q[6] - q[4]),
        0.5*(q[7] - q[3]),
        0.5*(q[2] - q[6]),
        0.5*(q[3] - q[5]),
        0.5*(q[4] - q[2])],
        ])

def dJ(q, dq):
    """Returns an m x n matrix of derivatives dJ/dt. This would technically be
    a rank 3 tensor, but can be represented by the mxn p(pC/pq dq/dt)/pq"""
    if q[6] >= wall:
        return np.array([[
            0.5*(dq[3] - dq[5]),
            0.5*(dq[4] - dq[2]),
            0.5*(dq[5] - dq[1]),
            0.5*(dq[0] - dq[4]),
            0.5*(dq[1] - dq[3]),
            0.5*(dq[2] - dq[0]),
            0, 0],

            [0, 0, 0, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, 0, 0, 0],

            [0, 0,
            0.5*(dq[5] - dq[7]),
            0.5*(dq[6] - dq[4]),
            0.5*(dq[7] - dq[3]),
            0.5*(dq[2] - dq[6]),
            0.5*(dq[3] - dq[5]),
            0.5*(dq[4] - dq[2])],

            [0, 0, 0, 0, 0, 0, 0, 0],
        ])
    return np.array([[
        0.5*(dq[3] - dq[5]),
        0.5*(dq[4] - dq[2]),
        0.5*(dq[5] - dq[1]),
        0.5*(dq[0] - dq[4]),
        0.5*(dq[1] - dq[3]),
        0.5*(dq[2] - dq[0]),
        0, 0],

        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],

        [0, 0,
        0.5*(dq[5] - dq[7]),
        0.5*(dq[6] - dq[4]),
        0.5*(dq[7] - dq[3]),
        0.5*(dq[2] - dq[6]),
        0.5*(dq[3] - dq[5]),
        0.5*(dq[4] - dq[2])],
    ])

qs = np.empty((simulation_steps+1, num_nodes*dim))
dqs = np.empty((simulation_steps+1, num_nodes*dim))
qs[0] = q0
dqs[0] = dq0

for idx in range(simulation_steps):
    q = qs[idx]
    dq = dqs[idx]
    J = jacobian(q)
    lagrange_mult = (
        np.linalg.inv(J @ W @ J.T)
        @ ((J @ W @ B - dJ(q, dq)) @ dq - J @ W @ Q[idx] - ks * C(q) - kd * J @ dq)
    )


    reactions = J.T @ lagrange_mult
    ddq = W @ (Q[idx] + reactions - B @ dq)
    dqs[idx+1] = dq + ddq * dt
    qs[idx+1] = q + dq * dt


In [ ]:
%matplotlib widget
import matplotlib.pyplot as plt
from matplotlib import animation

def get_triangle_coords(q):
    return ([q[0], q[2], q[4], q[0]], [q[1], q[3], q[5], q[1]])

def get_net_coords(q):
    return ([q[0], q[2], q[4], q[0], q[4], q[6], q[2]], [q[1], q[3], q[5], q[1], q[5], q[7], q[3]])

fig, ax = plt.subplots()
ax.axvline(1.5)
ax.set_aspect('equal')
ax.set_xlim([-5, 5])
ax.set_ylim([-5, 5])
# tri, = ax.plot(*get_triangle_coords(qs[0]))
net, = ax.plot(*get_net_coords(qs[0]))
arrow = ax.arrow(qs[0,4], qs[0,5], Q[0,4], Q[0,5], length_includes_head=True, head_width=.1)


def update(frame):
    # tri.set_data(*get_triangle_coords(qs[frame]))
    net.set_data(*get_net_coords(qs[frame]))
    arrow.set_data(x=qs[frame, 4], y=qs[frame, 5], dx=Q[frame, 4], dy=Q[frame,5])
    return net, arrow,

ani = animation.FuncAnimation(fig, update, frames=simulation_steps, interval=int(dt*1000))
# ani.save('triangle_animation.mp4')
